In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
PATH = '/content/gdrive/MyDrive/데분/데이콘/주차수요/data/'

In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df  = pd.read_csv(PATH+'train.csv')
test_df  = pd.read_csv(PATH+'test.csv')
info_df  = pd.read_csv(PATH+'age_gender_info.csv')

In [ ]:
train_df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
train_df.shape

(2952, 15)

In [ ]:
test_df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [ ]:
test_df.shape

(1022, 14)

In [ ]:
info_df.head()

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,0.072613,0.087149,0.072146,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,0.067925,0.085751,0.068819,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,경기도,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,전라북도,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


In [ ]:
info_df.shape

(16, 23)

In [ ]:
train_df.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [ ]:
train_df.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test_df.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [ ]:
train_df[['임대보증금', '임대료']] = train_df[['임대보증금', '임대료']].fillna(0)
test_df[['임대보증금', '임대료']] = test_df[['임대보증금', '임대료']].fillna(0)

In [ ]:
subway_null_codes = train_df[train_df['지하철'].isnull()].단지코드.unique()
train_df.loc[train_df.단지코드.isin(subway_null_codes), '지하철'].value_counts(dropna=False)

NaN    211
Name: 지하철, dtype: int64

In [ ]:
subway_null_codes = test_df[test_df['지하철'].isnull()].단지코드.unique()
test_df.loc[test_df.단지코드.isin(subway_null_codes), '지하철'].value_counts(dropna=False)

NaN    42
Name: 지하철, dtype: int64

In [ ]:
bus_null_codes = train_df[train_df['버스'].isnull()].단지코드.unique()
train_df.loc[train_df.단지코드.isin(bus_null_codes), '버스'].value_counts(dropna=False)

NaN    4
Name: 버스, dtype: int64

In [ ]:
cols = ['지하철', '버스']
train_df[cols] = train_df[cols].fillna(0)
test_df[cols] = test_df[cols].fillna(0)

In [ ]:
test_df.loc[test_df.단지코드.isin(['C2411']) & test_df.신분.isnull(), '신분'] = 'A'

In [ ]:
test_df.loc[test_df.단지코드.isin(['C2253']) & test_df.신분.isnull(), '신분'] = 'C'

In [ ]:
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [ ]:
train_df['전용면적'] = train_df['전용면적']//5*5
test_df['전용면적'] = test_df['전용면적']//5*5

In [ ]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train_df['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(train_df['단지코드'].unique())):
    temp = train_df[train_df['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test_df['단지코드'].unique())):
    temp = test_df[test_df['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:11, 35.32it/s]
150it [00:04, 35.80it/s]


In [ ]:
new_train.head()

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_10.0,면적_15.0,면적_20.0,면적_315.0,면적_60.0,면적_135.0,면적_75.0,면적_105.0,면적_580.0,면적_125.0,면적_240.0,면적_400.0,면적_245.0,면적_405.0,면적_80.0,면적_65.0,등록차량수
0,C2483,900.0,38.0,경상북도,1425.0,0.0,3.0,149.0,665.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,17.0,경상남도,624.0,0.0,3.0,80.0,132.0,0.0,276.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,13.0,대전광역시,1285.0,1.0,1.0,0.0,62.0,0.0,390.0,170.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,6.0,경기도,734.0,1.0,3.0,120.0,303.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,14.0,전라북도,645.0,0.0,2.0,254.0,246.0,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0


In [ ]:
info_df2=info_df.copy()

In [ ]:
info_df2['주차가능']=info_df['10대미만(남자)']+info_df['10대미만(여자)']+info_df['10대(남자)']+info_df['10대(여자)']+info_df['20대(남자)']+info_df['20대(여자)']+info_df['80대(남자)']+info_df['80대(여자)']+info_df['90대(남자)']+info_df['90대(여자)']+info_df['100대(남자)']+info_df['100대(여자)']
info_df2['주차불가능']=info_df['30대(남자)']+info_df['30대(여자)']+info_df['40대(남자)']+info_df['40대(여자)']+info_df['50대(남자)']+info_df['50대(여자)']+info_df['60대(남자)']+info_df['60대(여자)']+info_df['70대(남자)']+info_df['70대(여자)']

In [ ]:
info_df2=info_df2[['지역','주차가능']]
info_df2.head()

,지역,주차가능
0,경상북도,0.356965
1,경상남도,0.343506
2,대전광역시,0.320455
3,경기도,0.347395
4,전라북도,0.370487


In [ ]:
new_train=pd.merge(new_train,info_df2,how='left',on='지역')
new_test=pd.merge(new_test,info_df2,how='left',on='지역')

ValueError: ignored

In [ ]:
local_map = {}
for i, loc in enumerate(new_train['지역'].unique()):
    local_map[loc] = i

In [ ]:
new_train['지역'] = new_train['지역'].map(local_map)
new_test['지역'] = new_test['지역'].map(local_map)

In [ ]:
new_train.head()

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_10.0,면적_15.0,면적_20.0,면적_315.0,면적_60.0,면적_135.0,면적_75.0,면적_105.0,면적_580.0,면적_125.0,면적_240.0,면적_400.0,면적_245.0,면적_405.0,면적_80.0,면적_65.0,등록차량수,주차가능
0,C2483,900.0,38.0,0,1425.0,0.0,3.0,149.0,665.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0,0.356965
1,C2515,545.0,17.0,1,624.0,0.0,3.0,80.0,132.0,0.0,276.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0,0.343506
2,C1407,1216.0,13.0,2,1285.0,1.0,1.0,0.0,62.0,0.0,390.0,170.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0,0.320455
3,C1945,755.0,6.0,3,734.0,1.0,3.0,120.0,303.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0,0.347395
4,C1470,696.0,14.0,4,645.0,0.0,2.0,254.0,246.0,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0,0.370487


In [ ]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 위와 동일하지만 더 효율적입니다(fit_transform)
x_scaled_d = scaler.fit_transform(x_train)

#해당 fit으로 test데이터도 transform 해줍니다
x_test_scaled = scaler.transform(x_test)

In [ ]:
model = LGBMRegressor(n_jobs=-1, random_state=0)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=0)

In [ ]:
param_grid = {
    'num_leaves': [16, 31, 62, 127],
    'reg_alpha': [0.1,0.25, 0.5],
    'min_data_in_leaf': [15, 30, 50, 100, 300, 400],
    'lambda_l1': [0, 0.5, 1, 1.5],
    'lambda_l2': [0, 0.5, 1],
    'gamma': [0.0001, 0.01, 0.1, 1, 10]
    }

grid = GridSearchCV(model, param_grid=param_grid, 
                    scoring=['neg_mean_absolute_error'],refit='neg_mean_absolute_error',
                    return_train_score=True, cv=3)
grid.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=0,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     su...
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'gamma': [0.0001, 0.01, 0.1, 1, 10],
                         'lambda_l1': [0, 0.5, 1, 1.5],
                         'lambda_l2': [0, 0.5, 1],
                         'min_data_in_leaf': [15,

In [ ]:
grid.score(x_valid,y_valid)

-144.77273949017012

In [ ]:
pred = grid.predict(x_test_scaled)

In [ ]:
submission  = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
submission['num'] = pred

In [ ]:
submission.to_csv(PATH+'ver4_3.csv', index=False)